<a href="https://colab.research.google.com/github/amitsangani/llama/blob/main/Llama_2_13b_chat_and_llama_cpp_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# llama-cpp-python



* `llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook.
* llama-cpp-python package provides simple Python bindings for llama.cpp library.
* `GGML` is a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware.

If you want to use only the CPU, you can replace the content of the cell below with the following lines. The library works the same with a CPU, but the inference can take about three times longer compared to using it on a GPU.
```
# CPU llama-cpp-python
!pip install llama-cpp-python
```

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

In [ ]:
# For download the models
!pip install huggingface_hub

# Select the "Llama 2 13B-chat" model


In Colab with T4 GPU, we can run models of up to 20B of parameters with all optimizations, but this may degrade the quality of the model's inference. In this case, we will use a [Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat)

# Model quantization

We can quantize the model using this library, but for practical purposes, it is
better to use pre-quantized models. The resulting model is only compatible with libraries that support GGML.


```
# obtain the original LLaMA model weights and place them in ./models
ls ./models
65B 30B 13B 7B tokenizer_checklist.chk tokenizer.model

# install Python dependencies
python3 -m pip install -r requirements.txt

# convert the 7B model to ggml FP16 format
python3 convert.py models/7B/

# quantize the model to 4-bits (using q4_0 method)
./quantize ./models/7B/ggml-model-f16.bin ./models/7B/ggml-model-q4_0.bin q4_0

# run the inference
./main -m ./models/7B/ggml-model-q4_0.bin -n 128
```



#  Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

In [5]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

First, we download the model

In [6]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# Inference with llama-cpp-python

Loading the model

In [ ]:
# GPU
from llama_cpp import Llama
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )


For run in CPU
```
# CPU
from llama_cpp import Llama

lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    )
```



In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

We will use this prompt.

In [9]:
prompt = "What is the meaning of life?"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

Generating response

If you only use CPU, the response can take a long time. You can reduce the max_tokens to get a faster response.

In [10]:
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=256,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    echo=True
    )

print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: What is the meaning of life?

ASSISTANT:

The meaning of life is a question that has puzzled philosophers and theologians for centuries, and there is no one definitive answer. However, here are some possible perspectives on what might give our lives meaning:

1. Personal fulfillment: Some people believe that the purpose of life is to find personal fulfillment and happiness. This could involve pursuing our passions and interests, cultivating strong relationships with others, and taking care of ourselves physically and emotionally.
2. Religious or spiritual beliefs: Many religious and spiritual traditions offer a sense of meaning and purpose that is derived from a higher power or divine being. For example, some believe that our lives have a specific purpose or mission that is ordained by God or the universe.
3. Contribution to society: Some people find meaning in contributing to the greater goo

# Inference with langchain




In [ ]:
!pip -q install langchain

We can use the model that we loaded earlier. However, for illustrative purposes, we will load one from the 'langchain' library. Due to vRAM limitations, before running these cells, you need to delete the previous model from memory.

In [12]:
lcpp_llm.reset()
lcpp_llm.set_cache(None)
lcpp_llm = None
del lcpp_llm


Prompt for the model.

In [13]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download

template = """USER: {question}
ASSISTANT: Let's work this out in a step by step way to be sure we have the right answer."""
prompt = PromptTemplate(template=template, input_variables=["question"])

Stream tokens

In [14]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

Load the model

In [ ]:
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Loading model,
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=1024,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,
)

Generating response

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What is the meaning of life?"

llm_chain.run(question)

# References

- [renenyffenegger.ch - LLaMA C++ Library](https://renenyffenegger.ch/notes/development/Artificial-intelligence/language-model/LLM/LLaMA/libs/llama_cpp/)
- [LLaMA C++ Library Documentation](https://llama-cpp-python.readthedocs.io/en/latest/)
- [MacOS Install with Metal GPU - LLaMA C++ Library Documentation](https://llama-cpp-python.readthedocs.io/en/latest/install/macos/)
